In [1]:
from torchvision.datasets import ImageFolder
import numpy as np
import torch


# 设置随机数生成器种子
seed = 24
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# 数据集根目录
data_root = './data/dataset_new/tongue_color'
dataset = ImageFolder(root=data_root)

class_names = dataset.classes

print(len(dataset))
print(dataset.class_to_idx)
print(dataset[0])

92
{'dan_bai': 0, 'dan_hong': 1, 'jiang_she': 2, 'qing_zi': 3}
(<PIL.Image.Image image mode=RGB size=1664x1452 at 0x28F65EC3010>, 0)


In [2]:
from torchvision.transforms import transforms
from torch.utils.data import DataLoader


dataset_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

data_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
data_loader.dataset.transform = dataset_transform
# 获取所有图像数据
all_images = None

for inputs, _ in data_loader:
    all_images = inputs

# 计算数据集的均值和标准差
mean = torch.mean(all_images, dim=(0, 2, 3))
std = torch.std(all_images, dim=(0, 2, 3))

print(f"均值：{mean}")
print(f"标准差：{std}")

均值：tensor([0.4791, 0.4038, 0.4168])
标准差：tensor([0.2908, 0.2599, 0.2675])


In [3]:
from torch.utils.data import random_split
from imblearn.over_sampling import RandomOverSampler
from torch.utils.data import TensorDataset


class Cutout(object):
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)

        for _ in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1:y2, x1:x2] = 0

        mask = torch.from_numpy(mask)
        img *= mask.unsqueeze(0)

        return img


# 数据增强和预处理
data_augmentation_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomApply(transforms.GaussianBlur(3, sigma=(0.1, 2.0)), 0.5),
    Cutout(n_holes=8, length=32),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])


# # 计算每个类别的样本数量
# class_counts = torch.bincount(torch.tensor(dataset.targets))

# # 计算每个类别的权重
# class_weights = 1 / class_counts.float()


# 划分数据集
train_size = int(4 / 6.0 * len(dataset))
val_size = int(1 / 6.0 * len(dataset))
test_size = len(dataset) - train_size - val_size
remaining_size = len(dataset) - train_size
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size])

train_dataset.dataset.transform = data_augmentation_transform
val_dataset.dataset.transform = transform
test_dataset.dataset.transform = transform

# 获取训练集的特征和标签
X_train = []
y_train = []
for inputs, labels in train_dataset:
    X_train.append(inputs.numpy())  # 将张量转换为 numpy 数组
    y_train.append(labels)
X_train = np.array(X_train)
y_train = np.array(y_train)

# 创建 RandomOverSampler 实例
ros = RandomOverSampler(random_state=24)

# 进行过采样
X_resampled, y_resampled = ros.fit_resample(
    X_train.reshape(X_train.shape[0], -1), y_train)

# 将特征转换回张量形式
X_resampled = torch.tensor(X_resampled.reshape(-1, *inputs.shape))

# 创建过采样后的训练集
oversampled_train_dataset = TensorDataset(
    X_resampled, torch.tensor(y_resampled))

# 创建 DataLoader
train_loader = DataLoader(oversampled_train_dataset,
                          batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [4]:
# 输出数据集大小
print("过采样后的训练集大小:", len(oversampled_train_dataset))
print("验证集大小:", len(val_dataset))
print("测试集大小:", len(test_dataset))

for inputs, labels in train_loader:
    print(inputs.shape, labels.shape)
    print(inputs[0])
    break

for inputs, labels in val_loader:
    print(inputs.shape, labels.shape)
    print(inputs[0])
    break

id2label = {id: label for id, label in enumerate(
    train_dataset.dataset.classes)}
label2id = {label: id for id, label in id2label.items()}

过采样后的训练集大小: 148
验证集大小: 15
测试集大小: 16
torch.Size([8, 3, 224, 224]) torch.Size([8])
tensor([[[-1.6473, -1.6473, -1.6473,  ..., -1.6473, -1.6473, -1.6473],
         [-1.6473, -1.6473, -1.6473,  ..., -1.6473, -1.6473, -1.6473],
         [-1.6473, -1.6473, -1.6473,  ..., -1.6473, -1.6473, -1.6473],
         ...,
         [-1.6473, -1.6473, -1.6473,  ..., -1.6473, -1.6473, -1.6473],
         [-1.6473, -1.6473, -1.6473,  ..., -1.6473, -1.6473, -1.6473],
         [-1.6473, -1.6473, -1.6473,  ..., -1.6473, -1.6473, -1.6473]],

        [[-1.5536, -1.5536, -1.5536,  ..., -1.5536, -1.5536, -1.5536],
         [-1.5536, -1.5536, -1.5536,  ..., -1.5536, -1.5536, -1.5536],
         [-1.5536, -1.5536, -1.5536,  ..., -1.5536, -1.5536, -1.5536],
         ...,
         [-1.5536, -1.5536, -1.5536,  ..., -1.5536, -1.5536, -1.5536],
         [-1.5536, -1.5536, -1.5536,  ..., -1.5536, -1.5536, -1.5536],
         [-1.5536, -1.5536, -1.5536,  ..., -1.5536, -1.5536, -1.5536]],

        [[-1.5580, -1.5580, -1.5580

In [5]:
from transformers import AutoFeatureExtractor


# 加载 Vision Transformer 的特征提取器
model_name = 'microsoft/swin-base-patch4-window7-224'
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_name,
    do_normalize=False,
    do_resize=False,
    do_rescale=False,
)
# feature_extractor.image_mean = mean
# feature_extractor.image_std = std
feature_extractor

c:\Users\ChenXi\.conda\envs\2.2.1_11.8\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTFeatureExtractor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "resample",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "do_normalize": false,
  "do_rescale": false,
  "do_resize": false,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [6]:
from torch.utils.data.dataset import Dataset


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]


def custom_collate(batch):

    # 将样本数据列表拆分为输入和标签
    images, labels = list(zip(*batch))
    inputs = feature_extractor(images, return_tensors='pt')

    inputs['labels'] = torch.tensor(labels)
    inputs['pixel_values'] = torch.stack(
        [input for input in inputs["pixel_values"]])
    return inputs


train_data = MyDataset(train_dataset)
val_data = MyDataset(val_dataset)
test_data = MyDataset(test_dataset)

In [7]:
from transformers import SwinForImageClassification

model = SwinForImageClassification.from_pretrained(model_name, num_labels=len(dataset.classes), id2label=id2label, label2id=label2id, ignore_mismatched_sizes = True)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.metrics import accuracy_score


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))
    # logits, labels = eval_pred
    # predictions = logits.argsort(axis=-1)[:, -5:]  # 获取top-5预测结果
    # # 将标签扩展为与预测结果相同的形状
    # labels_expanded = labels.reshape(-1, 1).repeat(5, axis=1)
    # correct_predictions = (predictions == labels_expanded)  # 判断预测结果是否包含正确标签
    # top5_accuracy = correct_predictions.any(axis=-1).mean()  # 计算top-5准确率
    # return {"accuracy": top5_accuracy}

In [9]:
from transformers import TrainingArguments, Trainer


def train(train_dataset: Dataset,
          valid_dataset: Dataset,
          pre_model_path: str
          ) -> None:
    """[summary]

    Args:
        train_dataset (Dataset): [训练集]
        valid_dataset (Dataset): [验证集]
        pre_model_path (str): [预训练模型文件夹]

    Returns:
        [type]: [description]
    """

    args = TrainingArguments(
        # report_to="wandb",
        output_dir=f"F:/model_checkpoint/tongue-color-classification",
        remove_unused_columns=False,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=30,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        logging_dir='logs',
        warmup_ratio=0.1,
        logging_steps=10,
        seed=seed,
    )

    # 每次训练都要重新载入模型，定义一个函数返回初始化的预训练模型
    def model_init():
        return SwinForImageClassification.from_pretrained(pre_model_path, num_labels=len(dataset.classes), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

    # 定义一个trainer
    trainer = Trainer(
        # 注意此处不是model
        model_init=model_init,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=custom_collate,
        compute_metrics=compute_metrics,
        tokenizer=feature_extractor,
    )

    # 超参数搜索范围
    def hp_space(trial):
        return {
            "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
            "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.3),
            "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 30),
            "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32]),
        }

    # trainer.hyperparameter_search方法进行超参数搜索
    best_trial = trainer.hyperparameter_search(
        direction="maximize",
        backend="optuna",
        n_trials=10,
        hp_space=hp_space
    )

    print("*************************************")
    print(" Best run %s" % str(best_trial))
    print("*************************************")

In [10]:
# Train and save results
train_results = train(train_data, val_data, model_name)


c:\Users\ChenXi\.conda\envs\2.2.1_11.8\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
You should probably TRAIN this model on a down-

  0%|          | 0/272 [00:00<?, ?it/s]

{'loss': 1.4424, 'grad_norm': 33.74838638305664, 'learning_rate': 1.0096524908660242e-05, 'epoch': 0.62}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.9174505472183228, 'eval_accuracy': 0.7333333333333333, 'eval_runtime': 0.4139, 'eval_samples_per_second': 36.241, 'eval_steps_per_second': 4.832, 'epoch': 1.0}
{'loss': 1.0664, 'grad_norm': 20.912220001220703, 'learning_rate': 2.0193049817320484e-05, 'epoch': 1.25}
{'loss': 1.0345, 'grad_norm': 43.496665954589844, 'learning_rate': 2.8038546221754833e-05, 'epoch': 1.88}


...

{'eval_loss': 0.976004958152771, 'eval_accuracy': 0.7333333333333333, 'eval_runtime': 0.4148, 'eval_samples_per_second': 36.162, 'eval_steps_per_second': 4.822, 'epoch': 7.0}
{'loss': 0.3694, 'grad_norm': 89.93527221679688, 'learning_rate': 1.1513580595548858e-05, 'epoch': 7.5}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.9619613885879517, 'eval_accuracy': 0.7333333333333333, 'eval_runtime': 0.4118, 'eval_samples_per_second': 36.424, 'eval_steps_per_second': 4.857, 'epoch': 8.0}


[I 2024-05-06 17:55:47,246] Trial 9 pruned. 


*************************************
 Best run BestRun(run_id='1', objective=0.7333333333333333, hyperparameters={'learning_rate': 1.9368589239262595e-05, 'weight_decay': 0.21536378501859094, 'num_train_epochs': 9, 'per_device_train_batch_size': 8}, run_summary=None)
*************************************


In [17]:
train_results.


***** train metrics *****
  epoch                    =       30.0
  train_loss               =     0.1044
  train_runtime            = 0:01:48.07
  train_samples_per_second =     16.933
  train_steps_per_second   =      2.221


In [ ]:

# Evaluate on validation set
metrics = trainer.evaluate(val_data)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [12]:
# from timm.models import create_model

# Swin = create_model('swin_large_patch4_window7_224_in22k',pretrained=True)

In [13]:
import timm

model_names = timm.list_models(pretrained=True)
print(model_names)

['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k', 'beit_large_patch16_224.in22k_ft_in22k_in1k', 'beit_large_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_512.in22k_ft_in22k_in1k', 'beitv2_base_patch16_224.in1k_ft_in1k', 'beitv2_base_patch16_224.in1k_ft_in22k', 'beitv2_base_patch16_224.in1k_ft_in22k_in1k', 'beitv2_large_patch16_224.in1k_ft_in1k', 'beitv2_large_patch16_224.in1k_ft_in22k', 'beitv2_large_patch16_224.in1k_ft_in22k_in1k', 'botnet26t_256.c1_in1k', 'caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_b36.sail_in22k', 'caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_m36.sail_in22k', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s